In [2]:
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
import sklearn as sk
import dtale
import re
%config Completer.use_jedi = False

In [2]:
# df = pd.read_csv("donors.csv", low_memory=False, dtype=dtypes).drop(columns="Unnamed: 0")
# df.to_pickle("donors.pkl")
# sample = df.iloc[:1000, :].copy()
# sample.to_pickle("sample.pkl")

In [3]:
df = pd.read_pickle("donors.pkl")
sample = pd.read_pickle("sample.pkl")

In [4]:
dtypes = df.dtypes.to_dict()
types_dict = {
    "TCODE": str,
}
dtypes.update(types_dict)

# Variables

In [5]:
# Categorical variables
cat_vars = [
    "STATE", # category
    
    "NOEXCH", # binary
    "RECINHSE", # binary
    "RECP3", # binary
    "RECPGVG", # binary
    "RECSWEEP", # binary
    
    "MAJOR", # binary
    "MDMAUD", # category
    "MDMAUD_R", # category
    "MDMAUD_F", # category
    "MDMAUD_A", # category
    
    "DOMAIN", # category
    "HOMEOWNR", # binary
    "CHILD03", # category
    "CHILD07", # category
    "CHILD12", # category
]

# Continuous variables
cont_vars = [
    "AGE", # 
    "",
]

# Variables for plots
plot_vars = [
    "STATE", 
    "ZIP", # clean zip [TODO]
    "GEOCODE2", 
    "ODATEDW", # turn into year-month-01 [TODO]
]

# not used features
drop_vars = [
    "GEOCODE", # 80k nan
    "MAILCODE", 
    "OSOURCE", 
    "DATASRCE",
    "PVASTATE",
    "LIFESRC",
]

# features to process
fe_features = [
    "TCODE", # Shorten it [TODO]
    "DOB", # Create age groups [DONE]
    "ODATEDW" # Create loyalty groups
]

# feature engineering
cat_fe_features = [
    "ageGroup", # category
    "loyaltyGroup", # category
]

cont_fe_features = [
    
]

# EDA DONE

In [6]:
# MAIL OFFERS
# The following variables indicate the number of
# known times the donor has responded to other
# types of mail order offers.

groupMailOffers = df.loc[:, "MBCRAFT":"PUBOPP"].columns.to_list()

# dtale.show(df[mailOrderOffers])

for col in groupMailOffers:
    df[col] = df[col].fillna(0).astype(np.int16)

In [7]:
searchName = "MDMAUD"
namesToGroup = [col for col in df.columns.to_list() if searchName in col]
print(namesToGroup)

['MDMAUD', 'MDMAUD_R', 'MDMAUD_F', 'MDMAUD_A']


In [8]:
for name in namesToGroup:
    setattr(df, name, df[name].str.replace("X", "").replace("", np.nan))

In [9]:
# MAJOR
namesToGroup = ["MAJOR"] + namesToGroup
df.MAJOR.replace({"X": True, " ": False}, inplace=True)

In [10]:
# RFA_2
groupRFA_2 = ["RFA_2", "RFA_2R", "RFA_2F", "RFA_2A"]
namesToGroup += groupRFA_2
df[namesToGroup].head(2)

,MAJOR,MDMAUD,MDMAUD_R,MDMAUD_F,MDMAUD_A,RFA_2,RFA_2R,RFA_2F,RFA_2A
0,False,NaN,NaN,NaN,NaN,L4E,L,4,E
1,False,NaN,NaN,NaN,NaN,L2G,L,2,G


In [11]:
## TODO: Create categorical orders

incomeCats = ["Low", "Medium", "High"]
incomeBins = [0, 3, 6, 10]

df["cat_INCOME"] = pd.cut(
    df.INCOME, bins=incomeBins, labels=incomeCats
)

df.drop(columns=["INCOME"], inplace=True)

for c in namesToGroup:
    df[c] = pd.Categorical(df[c])

In [12]:
toReorder = ["ADATE", "RFA", "RDATE", "RAMNT"]
reorderedCols = []
for i in range(3, 25):
    reorderedCols += [str(col) + "_" + str(i) for col in toReorder]
reorderedCols = ["ADATE_2"] + groupRFA_2 + reorderedCols

dictIdx = {col: df.columns.get_loc(col) for col in reorderedCols}

temp = df[dictIdx.keys()].copy()
df.drop(columns=dictIdx.keys(), inplace=True)

for key in dictIdx.keys():
    df[key] = temp[key]

In [13]:
# Converting date columns into datetimes
for c in (dateNames := [c for c in df.columns.values if ("DATE" in c)]) :
    df[c] = pd.to_datetime(df[c], infer_datetime_format=True)

In [14]:
# AGE RELATED
df.DOB = pd.to_datetime(df.DOB)
df["AGE"] = np.int16(2020 - pd.to_datetime(df.DOB).dt.year)
df.AGE = df.AGE.where(df.AGE != 0, np.nan)

# age_groups_labels = ["0-18", "18-30", "30-60", "60+"]
age_groups_labels = ["Minor", "Young worker", "Worker", "Retired"]
age_groups_values = [0, 18, 30, 60, np.inf]

df["ageGroup"] = pd.cut(df.AGE, age_groups_values, labels=age_groups_labels)

In [15]:
# LOYALTY GROUP
# years passed since first gift
df["loyaltyAge"] = np.int16(2020 - pd.to_datetime(df.ODATEDW).dt.year)
# Origin groups [loyalty measured in years]
origin_groups_labels = ["0-1", "1-2", "2-5", "5-10", "10-20", "20+"]
origin_groups_values = [0, 1, 2, 5, 10, 20, np.inf]

df["loyaltyGroup"] = pd.cut(
    df.loyaltyAge, bins=origin_groups_values, labels=origin_groups_labels
)

In [16]:
# Always keep this cell last before EDA WIP
sample.to_pickle("sample_eda_done.pkl")
df.to_pickle("df_eda_done.pkl")

# EDA WIP

In [8]:
# restart work from here
sample = pd.read_pickle("sample_eda_done.pkl")
df = pd.read_pickle("df_eda_done.pkl")

In [9]:
df.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,RDATE_23,RAMNT_23,ADATE_24,RFA_24,RDATE_24,RAMNT_24,AGE,ageGroup,loyaltyAge,loyaltyGroup
0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,,...,2014-08-01,11.0,2014-06-01,S4E,2014-06-01,9.0,63.0,Retired,11,10-20
1,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,,...,NaT,NaN,2014-06-01,F1E,NaT,NaN,48.0,Worker,6,5-10
2,2010-01-01,AMH,1,NC,27017,,,NaT,0,,...,2014-07-01,7.0,2014-06-01,S3D,2014-06-01,6.0,NaN,NaN,10,5-10
3,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,,...,NaT,NaN,NaT,,NaT,NaN,72.0,Retired,13,10-20
4,2006-01-01,,0,FL,33176,,,1940-01-01,0,X,...,NaT,NaN,2014-06-01,A3D,NaT,NaN,80.0,Retired,14,10-20


In [45]:
# Drop entries where
# df[df.ODATEDW < df.DOB]

In [47]:
tcodes="""
    000    = _
    001    = MR.
    001001 = MESSRS.
    001002 = MR. & MRS.
    002    = MRS.
    002002 = MESDAMES
    003    = MISS
    003003 = MISSES
    004    = DR.
    004002 = DR. & MRS.
    004004 = DOCTORS
    005    = MADAME
    006    = SERGEANT
    009    = RABBI
    010    = PROFESSOR
    010002 = PROFESSOR & MRS.
    010010 = PROFESSORS
    011    = ADMIRAL
    011002 = ADMIRAL & MRS.
    012    = GENERAL
    012002 = GENERAL & MRS.
    013    = COLONEL
    013002 = COLONEL & MRS.
    014    = CAPTAIN
    014002 = CAPTAIN & MRS.
    015    = COMMANDER
    015002 = COMMANDER & MRS.
    016    = DEAN
    017    = JUDGE
    017002 = JUDGE & MRS.
    018    = MAJOR
    018002 = MAJOR & MRS.
    019    = SENATOR
    020    = GOVERNOR
    021002 = SERGEANT & MRS.
    022002 = COLNEL & MRS.
    024    = LIEUTENANT
    026    = MONSIGNOR
    027    = REVEREND
    028    = MS.
    028028 = MSS.
    029    = BISHOP
    031    = AMBASSADOR
    031002 = AMBASSADOR & MRS.
    033    = CANTOR
    036    = BROTHER
    037    = SIR
    038    = COMMODORE
    040    = FATHER
    042    = SISTER
    043    = PRESIDENT
    044    = MASTER
    046    = MOTHER
    047    = CHAPLAIN
    048    = CORPORAL
    050    = ELDER
    056    = MAYOR
    059002 = LIEUTENANT & MRS.
    062    = LORD
    063    = CARDINAL
    064    = FRIEND
    065    = FRIENDS
    068    = ARCHDEACON
    069    = CANON
    070    = BISHOP
    072002 = REVEREND & MRS.
    073    = PASTOR
    075    = ARCHBISHOP
    085    = SPECIALIST
    087    = PRIVATE
    089    = SEAMAN
    090    = AIRMAN
    091    = JUSTICE
    092    = MR. JUSTICE
    100    = M.
    103    = MLLE.
    104    = CHANCELLOR
    106    = REPRESENTATIVE
    107    = SECRETARY
    108    = LT. GOVERNOR
    109    = LIC.
    111    = SA.
    114    = DA.
    116    = SR.
    117    = SRA.
    118    = SRTA.
    120    = YOUR MAJESTY
    122    = HIS HIGHNESS
    123    = HER HIGHNESS
    124    = COUNT
    125    = LADY
    126    = PRINCE
    127    = PRINCESS
    128    = CHIEF
    129    = BARON
    130    = SHEIK
    131    = PRINCE AND PRINCESS
    132    = YOUR IMPERIAL MAJEST
    135    = M. ET MME.
    210    = PROF.
"""

# print(
#     re.sub(
#         r'0*(?P<keepleft>\d+)\W*=\W*(?P<keep>.*)', 
#         r'"\g<keepleft>" : "\g<keep>",', 
#         tcodes
#     ).strip()
# )

In [48]:
dictTCODE = {
    "0" : "_",
    "1" : "MR.",
    "1001" : "MESSRS.",
    "1002" : "MR. & MRS.",
    "2" : "MRS.",
    "2002" : "MESDAMES",
    "3" : "MISS",
    "3003" : "MISSES",
    "4" : "DR.",
    "4002" : "DR. & MRS.",
    "4004" : "DOCTORS",
    "5" : "MADAME",
    "6" : "SERGEANT",
    "9" : "RABBI",
    "10" : "PROFESSOR",
    "10002" : "PROFESSOR & MRS.",
    "10010" : "PROFESSORS",
    "11" : "ADMIRAL",
    "11002" : "ADMIRAL & MRS.",
    "12" : "GENERAL",
    "12002" : "GENERAL & MRS.",
    "13" : "COLONEL",
    "13002" : "COLONEL & MRS.",
    "14" : "CAPTAIN",
    "14002" : "CAPTAIN & MRS.",
    "15" : "COMMANDER",
    "15002" : "COMMANDER & MRS.",
    "16" : "DEAN",
    "17" : "JUDGE",
    "17002" : "JUDGE & MRS.",
    "18" : "MAJOR",
    "18002" : "MAJOR & MRS.",
    "19" : "SENATOR",
    "20" : "GOVERNOR",
    "21002" : "SERGEANT & MRS.",
    "22002" : "COLNEL & MRS.",
    "24" : "LIEUTENANT",
    "26" : "MONSIGNOR",
    "27" : "REVEREND",
    "28" : "MS.",
    "28028" : "MSS.",
    "29" : "BISHOP",
    "31" : "AMBASSADOR",
    "31002" : "AMBASSADOR & MRS.",
    "33" : "CANTOR",
    "36" : "BROTHER",
    "37" : "SIR",
    "38" : "COMMODORE",
    "40" : "FATHER",
    "42" : "SISTER",
    "43" : "PRESIDENT",
    "44" : "MASTER",
    "46" : "MOTHER",
    "47" : "CHAPLAIN",
    "48" : "CORPORAL",
    "50" : "ELDER",
    "56" : "MAYOR",
    "59002" : "LIEUTENANT & MRS.",
    "62" : "LORD",
    "63" : "CARDINAL",
    "64" : "FRIEND",
    "65" : "FRIENDS",
    "68" : "ARCHDEACON",
    "69" : "CANON",
    "70" : "BISHOP",
    "72002" : "REVEREND & MRS.",
    "73" : "PASTOR",
    "75" : "ARCHBISHOP",
    "85" : "SPECIALIST",
    "87" : "PRIVATE",
    "89" : "SEAMAN",
    "90" : "AIRMAN",
    "91" : "JUSTICE",
    "92" : "MR. JUSTICE",
    "100" : "M.",
    "103" : "MLLE.",
    "104" : "CHANCELLOR",
    "106" : "REPRESENTATIVE",
    "107" : "SECRETARY",
    "108" : "LT. GOVERNOR",
    "109" : "LIC.",
    "111" : "SA.",
    "114" : "DA.",
    "116" : "SR.",
    "117" : "SRA.",
    "118" : "SRTA.",
    "120" : "YOUR MAJESTY",
    "122" : "HIS HIGHNESS",
    "123" : "HER HIGHNESS",
    "124" : "COUNT",
    "125" : "LADY",
    "126" : "PRINCE",
    "127" : "PRINCESS",
    "128" : "CHIEF",
    "129" : "BARON",
    "130" : "SHEIK",
    "131" : "PRINCE AND PRINCESS",
    "132" : "YOUR IMPERIAL MAJEST",
    "135" : "M. ET MME.",
    "210" : "PROF.",
}

In [57]:
df["cat_TCODE"] = df.TCODE.replace(to_replace=dictTCODE)

In [58]:
df.cat_TCODE.unique()

array(['_', 'MR.', 'MRS.', 'MS.', 'MISS', 'MR. & MRS.', 'SISTER', 'DR.',
       'MAJOR', '980', 'CAPTAIN', 'MSS.', '72', '22', 'COLONEL & MRS.',
       '23', '45', 'LIEUTENANT', 'DR. & MRS.', '30', 'COLONEL', '202',
       '136', 'REVEREND & MRS.', '96', 'SR.', 'M.', 'SERGEANT', 'DOCTORS',
       '39002', '61', 'CHAPLAIN', 'BROTHER', '228', 'CAPTAIN & MRS.',
       '6400', 'FATHER', '25', '21', '94', 'GENERAL', '58002', '134',
       'MAJOR & MRS.', 'COMMODORE', 'RABBI', '76', 'ELDER', 'REVEREND',
       '93', 'JUDGE', '7', 'MASTER', '24002', 'COLNEL & MRS.'],
      dtype=object)

In [61]:
# Drop columns ?
condition = (df.cat_TCODE.isin(dictTCODE.values()))
df.cat_TCODE.where(condition, "_", inplace=True)

In [86]:
df.cat_TCODE.value_counts()[:10]

_             41118
MR.           25691
MRS.          17006
MS.            8258
MR. & MRS.     1861
MISS            835
DR.             367
MSS.             71
COLONEL          45
DR. & MRS.       44
Name: cat_TCODE, dtype: int64

In [46]:
df.groupby(by="cat_TCODE", axis=0)["AGE"].median()

cat_TCODE
7                  86.0
21                 66.5
22                 79.0
23                 67.0
25                 70.0
30                 70.5
45                 62.0
61                  NaN
72                 66.0
76                  NaN
93                  NaN
94                 53.0
96                 60.0
134                 NaN
136                44.0
202                70.5
228                20.0
980                57.5
6400                NaN
24002              43.0
39002              63.0
58002              81.0
BROTHER             NaN
CAPTAIN            75.0
CAPTAIN & MRS.     60.0
CHAPLAIN           71.0
COLNEL & MRS.       NaN
COLONEL            77.0
COLONEL & MRS.     77.0
COMMODORE          68.0
DOCTORS            37.0
DR.                60.0
DR. & MRS.         61.0
ELDER              70.0
FATHER              NaN
GENERAL             NaN
JUDGE              76.0
LIEUTENANT         63.0
M.                 10.0
MAJOR              64.0
MAJOR & MRS.       76.5
MASTER

In [16]:
# Flags

flagsBinary = [
    "NOEXCH",  # binary
    "RECINHSE",  # binary
    "RECP3",  # binary
    "RECPGVG",  # binary
    "RECSWEEP",  # binary
    "MAJOR",  # binary
]

df[flagsBinary].replace({"X": True, " ": False, "0": False, "1": True})

,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MAJOR
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,True,True,False,False,False
...,...,...,...,...,...,...
95407,False,False,False,False,False,False
95408,False,False,False,False,False,False
95409,False,False,True,False,False,False
95410,False,True,False,False,False,False


In [86]:
# Family Aggregate Data - loc[:, "DOMAIN":"WEALTH1"]
# FAD_name

domainCats = [
    letter + str(i) for i in range(1, 4) for letter in ["U", "C", "S", "T", "R"]
]

domainCats.reverse()
domainCats.insert(4, "U4")

df["cat_FAD_domain"] = pd.Categorical(df.DOMAIN, categories=domainCats, ordered=True)

In [87]:
df.cat_FAD_domain

0        T2
1        S1
2        R2
3        R2
4        S2
         ..
95407    C2
95408    C1
95409    C3
95410    C1
95411    C1
Name: FAD_domain, Length: 95412, dtype: category
Categories (16, object): ['R3' < 'T3' < 'S3' < 'C3' ... 'T1' < 'S1' < 'C1' < 'U1']